### Exploring GPX files

<https://ocefpaf.github.io/python4oceanographers/blog/2014/08/18/gpx/>

In [ ]:
%load_ext skip_kernel_extension

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the qreat circle distance between two points
    on the earth (specifiecd in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat1 - lat2
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt())
    
    # Radius of Earth in Kilometers is 6361
    km = 6371 * c
    return km


In [ ]:
import os
from glob import glob
import gpxpy
from pandas import DataFrame

def load_run_data(gpx_path, filter=""):
    path = os.path.join(gpx_path, "**/*" + filter + ".gpx")
    gpx_files = glob(path, recursive=True)
    
    print("Files ({0}): -".format(len(gpx_files)))    
    run_data = []
    for file_idx, gpx_file in enumerate(gpx_files):
        print("  {0}".format(gpx_file))
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        # Loop through tracks
        for track_idx, track in enumerate(gpx.tracks):
            track_name = track.name
            track_time = track.get_time_bounds().start_time
            track_length = track.length_3d()
            track_duration = track.get_duration()
            track_speed = track.get_moving_data().max_speed
            
            for seg_idx, segment in enumerate(track.segments):
                segment_length = segment.length_3d()
                for point_idx, point in enumerate(segment.points):
                    run_data.append([file_idx, os.path.basename(gpx_file), track_idx, track_name, 
                                     track_time, track_length, track_duration, track_speed, 
                                     seg_idx, segment_length, point.time, point.latitude, 
                                     point.longitude, point.elevation, segment.get_speed(point_idx)])
    return run_data

In [ ]:
from IPython.core.display import HTML

data = load_run_data(gpx_path='./_Data/GPX/', filter='TimeFormatted')
df = DataFrame(data, columns=['File_Index', 'File_Name', 'Index', 'Name',
                              'Time', 'Length', 'Duration', 'Max_Speed',
                              'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                              'Point_Longitude', 'Point_Elevation', 'Point_Speed'])


In [ ]:
df.count()

In [ ]:
# Data can be loaded twice. Once from Current and again from Archived files.

# if "File_Index" in df:
#     df = df.drop('File_Index', axis=1)

# if "File_Name" in df:
#     df = df.drop('File_Name', axis=1)

# if "Index" in df:
#     df = df.drop('Index', axis=1)

# if "Name" in df:
#     df = df.drop('Name', axis=1)

# df = df.drop_duplicates()

# df = df.loc[df.shift() == df]

In [ ]:
df.count()

In [ ]:
HTML(df.head().to_html(max_cols=4))

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
cols = ['Time', 'Length', 'Duration', 'Max_Speed']
tracks = df[cols].copy()
tracks['Length'] /= 1e3
tracks.drop_duplicates(inplace=True)
tracks.head()

In [ ]:
tracks['Year'] = tracks['Time'].apply(lambda x: x.year)
tracks['Month'] = tracks['Time'].apply(lambda x: x.month)
tracks_grouped_month = tracks.groupby(['Year','Month'])
tracks_grouped_month.describe().head()

In [ ]:
import matplotlib.pyplot as plt

figsize=(7, 3.5)

tracks_grouped_month = tracks.groupby(['Year', 'Month'])
ax = tracks_grouped_month['Length'].sum().plot(kind='bar', figsize=figsize)
xlabels = [text.get_text() for text in  ax.get_xticklabels()]
ax.set_xticklabels(xlabels, rotation=70)
_ = ax.set_ylabel('Distance (km)')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

figsize=(7, 3.5)

tracks['Day'] = tracks['Time'].apply(lambda x: x.day)
tracks_grouped_days = tracks.groupby(['Year', 'Month', 'Day'])
ax = tracks_grouped_days['Length'].sum().plot(kind='bar', figsize=figsize)
xlabels = [text.get_text() for text in  ax.get_xticklabels()]
ax.set_xticklabels(xlabels, rotation=70)
_ = ax.set_ylabel('Distance (km)')

plt.show()

In [ ]:
import mplleaflet
import matplotlib.pyplot as plt

from datetime import datetime

d = datetime(2017,12,16)

munich_hols = df[df['Time'] > d]
munich_hols = munich_hols.dropna()

# munich_hols.head()
munich_hols.tail()

In [ ]:
import pandas as pd
df = df.set_index(pd.DatetimeIndex(df['Point_Time']))
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
# del df['Time']

import mplleaflet
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

if "File_Index" in df:
    dfg = df.groupby(['File_Index', df.index.day, 'Segment_Index', 'Index'])
else:
    dfg = df.groupby([df.index.day, df.index.hour, 'Segment_Index'])
    
# dfg.head()

In [ ]:
if "File_Index" in df:
    DFList = [group[1] for group in dfg]
    for d in DFList:
        d.to_csv("_OUTPUT/data_{0}.csv".format(d['File_Index'][0]))
        ax.plot(d['Point_Longitude'], d['Point_Latitude'], linestyle='-', linewidth=10, alpha=0.75)
else:
    DFList = [group[1] for group in df.groupby(df.index.day)]
    for d in DFList:
        ax.plot(d['Point_Longitude'], d['Point_Latitude'], linestyle='-', linewidth=10, alpha=0.75)

mplleaflet.show(fig=fig)

In [ ]:
OUTPUT_FOLDER = '_OUTPUT'
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

df.to_csv('_OUTPUT/data.csv')

In [ ]:
# calculate distances on surface of ellipsoid
from vincenty import vincenty

if "File_Index" in df:
    for d in DFList:
        d['lastLat'] = d['Point_Latitude'].shift(1)
        d['lastLong'] = d['Point_Longitude'].shift(1)
        d['dist(meters)'] = d.apply(lambda x: vincenty((x['Point_Latitude'], x['Point_Longitude']), (x['lastLat'], x['lastLong'])), axis = 1) * 1000.

        print('Total distance as summed between points in track: ' + str(sum(d['dist(meters)'][1:])*0.000621371) + ' mi')
        # The df['dist'][1:] above is because the "shift" sets the first lastLon,lastLat as NaN.
        # print('Comparing to total distance contained in track.name: ' + track.name)
        break


In [ ]:
%%skip True

import folium

OUTPUT_PATH = '_OUTPUT/fol/'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

i = 0
for d in DFList:
    mymap = folium.Map( location=[ d.Point_Latitude.mean(), d.Point_Longitude.mean() ], zoom_start=14)

    for coord in d[['Point_Latitude','Point_Longitude']].values:
        folium.CircleMarker(location=[coord[0],coord[1]], radius=1,color='red').add_to(mymap)

    i += 1
    mymap.save(os.path.join('_OUTPUT/fol', "fol_{0}.html".format(i)))

In [ ]:
# http://benalexkeen.com/resampling-time-series-data-with-pandas/

weekly_summary = pd.DataFrame()
weekly_summary['speed'] = df.Max_Speed.resample('W').mean()
weekly_summary['duration'] = df.Duration.resample('W').sum()

weekly_summary = weekly_summary.dropna(subset=['speed', 'duration'])

weekly_summary.head()

In [ ]:
daily_summary = pd.DataFrame()
daily_summary['speed'] = df.Max_Speed.resample('D').mean()
daily_summary['duration'] = df.Duration.resample('D').sum()
daily_summary['length'] = df.Length.resample('D').sum()

daily_summary = daily_summary.dropna(subset=['speed', 'duration', 'length'])

daily_summary.head()

In [ ]:
import seawater as sw
from datetime import datetime

d1 = datetime(2018,3,29)
d2 = datetime(2018,3,30)

# ne66_hols = df[df['Point_Time'] >= d1]

# mask = (df['Point_Time'] > d1) & (df['Point_Time'] < d2)
# mask = (df['File_Index'] == 1) & (df['Point_Time'] > d1) & (df['Point_Time'] < d2)
mask = (df['Point_Time'] > d1) & (df['Point_Time'] < d2)
ne66_hols = df.loc[mask]

if 'File_Index' in ne66_hols:
    ne66_hols = ne66_hols.drop('File_Index', axis=1)

if 'File_Name' in ne66_hols:
    ne66_hols = ne66_hols.drop('File_Name', axis=1)

if 'Index' in ne66_hols:
    ne66_hols = ne66_hols.drop('Index', axis=1)

ne66_hols_nodups = ne66_hols.drop_duplicates()

dist, angles = sw.dist(ne66_hols_nodups['Point_Latitude'], ne66_hols_nodups['Point_Longitude'])

conversion_factor =  0.62137119223733

hols_dist_kms = dist.sum()
hols_dist_miles = hols_dist_kms * conversion_factor

ne66_hols_nodups.head()

print(f"{hols_dist_kms} km")
print(f"{hols_dist_miles} miles")